In [91]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import lightgbm as lgb

In [92]:
df1 = pd.read_csv("ford.csv")
df1["brand"] = "ford"
df2 = pd.read_csv("audi.csv")
df2["brand"] = "audi"
df3 = pd.read_csv("bmw.csv")
df3["brand"] = "bmw"
df4 = pd.read_csv("mercedes.csv")
df4["brand"] = "mercedes"
df5 = pd.read_csv("toyota.csv")
df5["brand"] = "toyata"
df6 = pd.read_csv("hyundai.csv")
df6["brand"] = "hyundai"
df = pd.concat([df1, df2, df3, df4, df5, df6])
df

,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand,tax(£)
0,Fiesta,2017,12000,Automatic,15944,Petrol,150.0,57.7,1.0,ford,NaN
1,Focus,2018,14000,Manual,9083,Petrol,150.0,57.7,1.0,ford,NaN
2,Focus,2017,13000,Manual,12456,Petrol,150.0,57.7,1.0,ford,NaN
3,Fiesta,2019,17500,Manual,10460,Petrol,145.0,40.3,1.5,ford,NaN
4,Fiesta,2019,16500,Automatic,1482,Petrol,145.0,48.7,1.0,ford,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4855,I30,2016,8680,Manual,25906,Diesel,NaN,78.4,1.6,hyundai,0.0
4856,I40,2015,7830,Manual,59508,Diesel,NaN,65.7,1.7,hyundai,30.0
4857,I10,2017,6830,Manual,13810,Petrol,NaN,60.1,1.0,hyundai,20.0
4858,Tucson,2018,13994,Manual,23313,Petrol,NaN,44.8,1.6,hyundai,145.0


In [93]:
model_list = df["model"].tolist()
model_list = list(dict.fromkeys(model_list))
model_list.sort()
model_list
fueltype_list = df["fueltype"].toList()
fueltype_list = list(dict.fromkeys(fueltype_list))
fueltype_list.sort()
fueltype_list

[' 1 Series',
 ' 2 Series',
 ' 3 Series',
 ' 4 Series',
 ' 5 Series',
 ' 6 Series',
 ' 7 Series',
 ' 8 Series',
 ' A Class',
 ' A1',
 ' A2',
 ' A3',
 ' A4',
 ' A5',
 ' A6',
 ' A7',
 ' A8',
 ' Accent',
 ' Amica',
 ' Auris',
 ' Avensis',
 ' Aygo',
 ' B Class',
 ' B-MAX',
 ' C Class',
 ' C-HR',
 ' C-MAX',
 ' CL Class',
 ' CLA Class',
 ' CLC Class',
 ' CLK',
 ' CLS Class',
 ' Camry',
 ' Corolla',
 ' E Class',
 ' EcoSport',
 ' Edge',
 ' Escort',
 ' Fiesta',
 ' Focus',
 ' Fusion',
 ' G Class',
 ' GL Class',
 ' GLA Class',
 ' GLB Class',
 ' GLC Class',
 ' GLE Class',
 ' GLS Class',
 ' GT86',
 ' Galaxy',
 ' Getz',
 ' Grand C-MAX',
 ' Grand Tourneo Connect',
 ' Hilux',
 ' I10',
 ' I20',
 ' I30',
 ' I40',
 ' I800',
 ' IQ',
 ' IX20',
 ' IX35',
 ' Ioniq',
 ' KA',
 ' Ka+',
 ' Kona',
 ' Kuga',
 ' Land Cruiser',
 ' M Class',
 ' M2',
 ' M3',
 ' M4',
 ' M5',
 ' M6',
 ' Mondeo',
 ' Mustang',
 ' PROACE VERSO',
 ' Prius',
 ' Puma',
 ' Q2',
 ' Q3',
 ' Q5',
 ' Q7',
 ' Q8',
 ' R Class',
 ' R8',
 ' RAV4',
 ' 

In [94]:
model_dictionary = dict(zip(model_list, list(range(134))))
model_dictionary


{' 1 Series': 0,
 ' 2 Series': 1,
 ' 3 Series': 2,
 ' 4 Series': 3,
 ' 5 Series': 4,
 ' 6 Series': 5,
 ' 7 Series': 6,
 ' 8 Series': 7,
 ' A Class': 8,
 ' A1': 9,
 ' A2': 10,
 ' A3': 11,
 ' A4': 12,
 ' A5': 13,
 ' A6': 14,
 ' A7': 15,
 ' A8': 16,
 ' Accent': 17,
 ' Amica': 18,
 ' Auris': 19,
 ' Avensis': 20,
 ' Aygo': 21,
 ' B Class': 22,
 ' B-MAX': 23,
 ' C Class': 24,
 ' C-HR': 25,
 ' C-MAX': 26,
 ' CL Class': 27,
 ' CLA Class': 28,
 ' CLC Class': 29,
 ' CLK': 30,
 ' CLS Class': 31,
 ' Camry': 32,
 ' Corolla': 33,
 ' E Class': 34,
 ' EcoSport': 35,
 ' Edge': 36,
 ' Escort': 37,
 ' Fiesta': 38,
 ' Focus': 39,
 ' Fusion': 40,
 ' G Class': 41,
 ' GL Class': 42,
 ' GLA Class': 43,
 ' GLB Class': 44,
 ' GLC Class': 45,
 ' GLE Class': 46,
 ' GLS Class': 47,
 ' GT86': 48,
 ' Galaxy': 49,
 ' Getz': 50,
 ' Grand C-MAX': 51,
 ' Grand Tourneo Connect': 52,
 ' Hilux': 53,
 ' I10': 54,
 ' I20': 55,
 ' I30': 56,
 ' I40': 57,
 ' I800': 58,
 ' IQ': 59,
 ' IX20': 60,
 ' IX35': 61,
 ' Ioniq': 62,
 ' K

In [95]:
from sklearn.preprocessing import LabelEncoder
Numerics = LabelEncoder()

In [96]:
inputs = df.drop(['tax(£)', 'tax', 'engineSize'], axis='columns')
target = df.price
target

0       12000
1       14000
2       13000
3       17500
4       16500
        ...  
4855     8680
4856     7830
4857     6830
4858    13994
4859    15999
Name: price, Length: 64131, dtype: int64

In [97]:
inputs = inputs.replace(model_dictionary)

In [98]:
inputs

,model,year,price,transmission,mileage,fuelType,mpg,brand
0,38,2017,12000,Automatic,15944,Petrol,57.7,ford
1,39,2018,14000,Manual,9083,Petrol,57.7,ford
2,39,2017,13000,Manual,12456,Petrol,57.7,ford
3,38,2019,17500,Manual,10460,Petrol,40.3,ford
4,38,2019,16500,Automatic,1482,Petrol,48.7,ford
...,...,...,...,...,...,...,...,...
4855,56,2016,8680,Manual,25906,Diesel,78.4,hyundai
4856,57,2015,7830,Manual,59508,Diesel,65.7,hyundai
4857,54,2017,6830,Manual,13810,Petrol,60.1,hyundai
4858,111,2018,13994,Manual,23313,Petrol,44.8,hyundai


In [99]:
model = linear_model.LinearRegression()

In [100]:
model.fit(inputs_n.values, target)

ValueError: could not convert string to float: ' Fiesta'

In [ ]:
linear_r2 = model.score(inputs_n.values, target)
linear_r2

0.43102025753715567

In [ ]:
pred=model.predict([[2016, 20000, 43.5, 14, 0, 0, 2]])
pred

ValueError: X has 7 features, but LinearRegression is expecting 6 features as input.

In [ ]:
lgb_model = lgb.LGBMRegressor(
    boosting_type='gbdt',
    num_leaves=31,
    n_estimators=100,
    reg_lambda=1.0
)

lgb_model.fit(inputs_n, target)

lgb_r2 = lgb_model.score(inputs_n.values, target)
lgb_r2

0.9491089317988514

In [ ]:
pred=lgb_model.predict([[2016, 11472, 57.6, 111, 0, 0, 3]])
pred

array([15331.27405719])